In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph,START,END
from langchain_core.runnables import RunnableConfig


In [2]:
class ChatState(TypedDict):
    user_input:str
    response: str

In [3]:
from langgraph.store.memory import InMemoryStore
store=InMemoryStore()

def memory_namespace(user_id:str):
    return (user_id,"profile")

In [4]:
def load_memory(state:ChatState,config):
    user_id=config["configurable"]["user_id"]
    memories=store.search(memory_namespace(user_id))

    if memories:
        last_memory=memories[-1].value
    else:
        last_memory={}
    
    return {
        "response":f"Loaded memory: {last_memory}"
    }

In [5]:
def chat_node(state:ChatState):
    user_input=state['user_input']

    if 'pizza' in user_input.lower():
        return {
            "response":"Nice I'll remember that"
        }
    return{
        "response":"Got it!"
    }

In [6]:
import uuid

def save_memory(state:ChatState,config):
    user_id=config['configurable']['user_id']
    text=state["user_input"]

    if "pizza" in text.lower() or 'football' in text.lower():
        store.put(
            memory_namespace(user_id),
            str(uuid.uuid4()),
            {"food_preference":"pizza"}
        )
    return {}

In [7]:
workflow=StateGraph(ChatState)

workflow.add_node("load_memory",load_memory)
workflow.add_node("chat",chat_node)
workflow.add_node("save_memory",save_memory)

workflow.add_edge(START,"load_memory")
workflow.add_edge("load_memory","chat")
workflow.add_edge("chat","save_memory")
workflow.add_edge("save_memory",END)


from langgraph.checkpoint.memory import InMemorySaver
checkpointer=InMemorySaver()
graph=workflow.compile(checkpointer=checkpointer)

In [8]:
# config:RunnableConfig={"configurable":{"thread_id":"1"}}
config={
    "configurable":{
        "thread_id":"thread-1",
        "user_id":"1"
    }
}

graph.invoke(
    {"user_input":"I like pizza","response":""},
    config
)

{'user_input': 'I like pizza', 'response': "Nice I'll remember that"}

In [9]:
config={"configurable":{"thread_id":"thread-1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-2851-6c40-8003-db1f59c93882'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.957496+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=('save_memory',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.956273+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284b-6e08-8001-c3f

In [19]:
config = {
    "configurable": {
        "thread_id": "thread-2",
        "user_id": "1"
    }
}

graph.invoke(
    {"user_input": "Hello", "response": ""},
    config=config
)


{'user_input': 'Hello', 'response': 'Got it!'}

In [13]:
config={"configurable":{"thread_id":"thread-2"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'user_input': 'Hello', 'response': 'Got it!'}, next=(), config={'configurable': {'thread_id': 'thread-2', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd210-3969-65d1-8003-0adb5c07b3b7'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:44:28.443372+00:00', parent_config={'configurable': {'thread_id': 'thread-2', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd210-3967-62dd-8002-7c7a0b9e39ff'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'Hello', 'response': 'Got it!'}, next=('save_memory',), config={'configurable': {'thread_id': 'thread-2', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd210-3967-62dd-8002-7c7a0b9e39ff'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:44:28.442477+00:00', parent_config={'configurable': {'thread_id': 'thread-2', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd210-3964-6ea1-8001-1185ac87e634'}}, tasks=(PregelTask(id='ae9beea9-8

In [15]:
memories=store.search(memory_namespace("1"))

In [16]:
memories

[Item(namespace=['1', 'profile'], key='ab77f34e-4335-48cb-ad06-a949b03f9d03', value={'food_preference': 'pizza'}, created_at='2026-01-29T14:43:11.754545+00:00', updated_at='2026-01-29T14:43:11.754546+00:00', score=None),
 Item(namespace=['1', 'profile'], key='df143e0a-1975-46a7-a9ac-7dbbf2f7f986', value={'food_preference': 'pizza'}, created_at='2026-01-29T14:43:52.391580+00:00', updated_at='2026-01-29T14:43:52.391587+00:00', score=None)]

In [20]:
graph.invoke(
    {"user_input":"I like football","response":""},
    config
)

{'user_input': 'I like football', 'response': 'Got it!'}

In [23]:
# config:RunnableConfig={"configurable":{"thread_id":"1"}}
config2={
    "configurable":{
        "thread_id":"thread-3",
        "user_id":"2"
    }
}

graph.invoke(
    {"user_input":"I like pizza","response":""},
    config2
)

{'user_input': 'I like pizza', 'response': "Nice I'll remember that"}

In [24]:
config2={"configurable":{"thread_id":"thread-1"}}
list(graph.get_state_history(config2))

[StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd21c-7ba2-63b2-800d-84cc2ea568e1'}}, metadata={'source': 'loop', 'step': 13, 'parents': {}, 'user_id': '2'}, created_at='2026-01-29T14:49:57.509804+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd21c-7b9d-6ced-800c-7d7f23c2992b'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=('save_memory',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd21c-7b9d-6ced-800c-7d7f23c2992b'}}, metadata={'source': 'loop', 'step': 12, 'parents': {}, 'user_id': '2'}, created_at='2026-01-29T14:49:57.507998+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd21c-7b9a-6723-800b-5

In [10]:
# config:RunnableConfig={"configurable":{"thread_id":"1"}}
config3={
    "configurable":{
        "thread_id":"thread-3",
        "user_id":"4"
    }
}

graph.invoke(
    {"user_input":"I like pizza","response":""},
    config3
)

{'user_input': 'I like pizza', 'response': "Nice I'll remember that"}

In [11]:
config3={"configurable":{"thread_id":"thread-1"}}
list(graph.get_state_history(config3))

[StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-2851-6c40-8003-db1f59c93882'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.957496+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=('save_memory',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.956273+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284b-6e08-8001-c3f

In [12]:
# config:RunnableConfig={"configurable":{"thread_id":"1"}}
config={
    "configurable":{
        "thread_id":"thread-3",
        "user_id":"10"
    }
}

graph.invoke(
    {"user_input":"I like pizza","response":""},
    config
)

{'user_input': 'I like pizza', 'response': "Nice I'll remember that"}

In [13]:
config={"configurable":{"thread_id":"thread-1"}}
list(graph.get_state_history(config))

[StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-2851-6c40-8003-db1f59c93882'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.957496+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'user_input': 'I like pizza', 'response': "Nice I'll remember that"}, next=('save_memory',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284e-6ca8-8002-0157425940c9'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}, 'user_id': '1'}, created_at='2026-01-29T14:57:51.956273+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f0fd22e-284b-6e08-8001-c3f